<a href="https://colab.research.google.com/github/Poulami3008/SYNC-Internship/blob/main/Building%20Chatbot/Building_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#defining the Intentions of a Chatbot
 {"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye", "Take Care"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thank you", "Thanks a lot", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?"],
     "responses": ["I'm Russel, your bot assistant", "I'm Russel, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["What is your name", "What should I call you", "What's your name?"],
    "responses": ["I'm Russel!", "Just call me as Russel"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "Give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "Support me please"],
    "responses": ["Tell me how can I assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "buildapp",
    "patterns": ["I need to build a weather app", "I want to build a weather app", "Can you build a weather app for me"],
    "responses": ["You can just easily build a weather app from our web site", "Just go to our web site and follow the instructions to build a weather app"]
    }
]
}

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'Take Care'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thank you',
    'Thanks a lot',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?'],
   'responses': ["I'm Russel, your bot assistant",
    "I'm Russel, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['What is your name',
    'What should I call you',
    "What's your name?"],
   'responses': ["I'm Russel!", 'Just call me as Russel']},
  {'tag': 'help',
   'patterns': ['Could you help me?',
    'Give me a hand please',
    'Can you help?',
    'What can you do f

In [ ]:
#importing the necessary libraries and packages
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
!pip install colorama

In [ ]:
#read the JSON file and process the required files
with open('/content/sample_data/intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [ ]:
#converting the categorical data into numerical data
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
#vectorize the data using the Tokenization method to create a chatbot
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
#building the model and train the data
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dense_8 (Dense)             (None, 7)                 119       
                                                                 
Total params: 16,663
Trainable params: 16,663
Non-trainable params: 0
__________________________________________________

In [ ]:
#to save the trained model
model.save("chat_model")

import pickle

#to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#let's build a chatbot with the trained machine-learning model
import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("/content/sample_data/intents.json") as file:
    data = json.load(file)


def chat():
    #load trained model
    model = keras.models.load_model('chat_model')

    #load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    #load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    #parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Hey
1/1 [==============================] - 0s 68ms/step
ChatBot: Hi
User: Who are you?
1/1 [==============================] - 0s 26ms/step
ChatBot: I'm Russel, an Artificial Intelligent bot
User: What should I call you?
1/1 [==============================] - 0s 26ms/step
ChatBot: Just call me as Russel
User: Can you help?
1/1 [==============================] - 0s 23ms/step
ChatBot: Yes Sure, How can I support you
User: I want to build a weather app
1/1 [==============================] - 0s 25ms/step
ChatBot: Just go to our web site and follow the instructions to build a weather app
User: Thanks a lot
1/1 [==============================] - 0s 31ms/step
ChatBot: You're most welcome!
User: quit
